<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%B4%D0%BE%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D1%80%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Дополненная реальность видео OpenCV


# OpenCV Video Augmented Reality 


В этом уроке вы узнаете, как использовать дополненную реальность в реальном времени в видео с помощью OpenCV.


In this tutorial, you will learn how to use real-time augmented reality in video using OpenCV.





OpenCV предназначена для работы с обработкой изображений в реальном времени, поэтому мы также можем использовать OpenCV для дополненной реальности в реальном времени.

OpenCV is designed to handle real-time image processing, so we can also use OpenCV for real-time augmented reality.





Цели этого урока:

Objectives of this lesson:

* Upload two videos (The first video will act as our "eyes" in the real world (ie what our camera sees). The second video we will place in the first using ArUco markers and a Pantone card.  Загрузить два видео (Первое видео будет действовать как наши «глаза» в реальном мир (то есть то, что видит наша камера). Второе видео мы поместим в первое с помощью маркеров ArUco и карты Pantone.
* Detect ArUco markers in every video frame. Обнаруженить маркеров ArUco в каждом кадре видео
* Apply perspective transform to the second video to match the original frame with the deformed frame, thus creating augmented reality. Применить перспективное преобразование для второго видео, чтобы сопоставить исходный кадр с кадром деформированным, создав таким образом дополненную реальность.



### Загрузка файлов
### Download the code zip file

In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_18.video_augmented_reality.zip
!unzip -qq Lesson_18.video_augmented_reality.zip
%cd /content/Lesson_18.video_augmented_reality/

--2022-02-12 10:28:09--  http://dataudt.ru/datasets/cv/Lesson_18.video_augmented_reality.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19563931 (19M) [application/zip]
Saving to: ‘Lesson_18.video_augmented_reality.zip’

Lesson_18.video_aug 100%[===================>]  18.66M  6.80MB/s    in 2.7s    

2022-02-12 10:28:12 (6.80 MB/s) - ‘Lesson_18.video_augmented_reality.zip’ saved [19563931/19563931]

/content/Lesson_18.video_augmented_reality


### Импорт библиотек
### Import Packages

In [2]:
# импортируем необходимые библиотеки
# import the necessary packages
from collections import deque
import numpy as np
import argparse
import imutils
import time
import cv2

### Реализация нашего детектора маркеров дополненной реальности
### Implementing our marker detector/augmented reality utility function

In [3]:
# очищаем кэш контрольных точек
# initialize our cached reference points
CACHED_REF_PTS = None

In [4]:
def find_and_warp(frame, source, cornerIDs, arucoDict, arucoParams,
	useCache=False):
    # получаем ссылку на кэш контрольных точек
	# grab a reference to our cached reference points
	global CACHED_REF_PTS

    # определим ширину и высоту кадра и исходного изображения соответственно
	# grab the width and height of the frame and source image,
	# respectively
	(imgH, imgW) = frame.shape[:2]
	(srcH, srcW) = source.shape[:2]

    # обнаруживаем маркеры AruCo во входном кадре
	# detect AruCo markers in the input frame
	(corners, ids, rejected) = cv2.aruco.detectMarkers(
		frame, arucoDict, parameters=arucoParams)

    # если мы *не* нашли наши четыре маркера ArUco, инициализируем
	# пустой список идентификаторов, в противном случае разворачиваем список идентификаторов
	# if we *did not* find our four ArUco markers, initialize an
	# empty IDs list, otherwise flatten the ID list
	ids = np.array([]) if len(corners) != 4 else ids.flatten()

    # инициализируем наш список контрольных точек
	# initialize our list of reference points
	refPts = []

    # перебираем все идентификаторы маркеров ArUco в верхнем левом, верхнем правом,
	# нижнем правом и нижнем левом углах
	# loop over the IDs of the ArUco markers in top-left, top-right,
	# bottom-right, and bottom-left order
	for i in cornerIDs:
        # берем индекс угла с текущим идентификатором
		# grab the index of the corner with the current ID
		j = np.squeeze(np.where(ids == i))

        # если вместо списка с целыми числами мы получаем пустой список,
		# то мы не смогли найти маркер с текущим ID
		# if we receive an empty list instead of an integer index,
		# then we could not find the marker with the current ID
		if j.size == 0:
			continue

        # в противном случае добавляем координаты угла (x, y) в наш список контрольных точек
		# otherwise, append the corner (x, y)-coordinates to our list
		# of reference points
		corner = np.squeeze(corners[j])
		refPts.append(corner)

    # проверяем, смогли ли мы найти четыре маркера ArUco
	# check to see if we failed to find the four ArUco markers
	if len(refPts) != 4:
        # если у нас есть кэш контрольных точек, возвращаемся к нему
		# if we are allowed to use cached reference points, fall
		# back on them
		if useCache and CACHED_REF_PTS is not None:
			refPts = CACHED_REF_PTS

        # если мы не сможем использовать кэш и / или нет
		# предыдущих кэшированных контрольные точки, то возвращаем None
		# otherwise, we cannot use the cache and/or there are no
		# previous cached reference points, so return early
		else:
			return None

    # если кэш доступен, то обновляем его с текущими данными
	# if we are allowed to use cached reference points, then update
	# the cache with the current set
	if useCache:
		CACHED_REF_PTS = refPts

    # распаковываем наши контрольные точки ArUco и используем их для
	# определения * назначения * матрицы преобразования, убедившись, что точки
	# указаны в верхнем левом, верхнем правом, нижнем правом и нижнем левом углах
	# unpack our ArUco reference points and use the reference points
	# to define the *destination* transform matrix, making sure the
	# points are specified in top-left, top-right, bottom-right, and
	# bottom-left order
	(refPtTL, refPtTR, refPtBR, refPtBL) = refPts
	dstMat = [refPtTL[0], refPtTR[1], refPtBR[2], refPtBL[3]]
	dstMat = np.array(dstMat)

    # определим матрицу преобразования для * исходного * изображения
	# в верхнем левом, верхнем правом, нижнем правом и нижнем левом углах
	# define the transform matrix for the *source* image in top-left,
	# top-right, bottom-right, and bottom-left order
	srcMat = np.array([[0, 0], [srcW, 0], [srcW, srcH], [0, srcH]])

    # вычисляем матрицу гомографии, а затем деформируем исходное изображение
	# до нужного виды, основываясь на матрице
	# compute the homography matrix and then warp the source image to
	# the destination based on the homography
	(H, _) = cv2.findHomography(srcMat, dstMat)
	warped = cv2.warpPerspective(source, H, (imgW, imgH))

    # создаем маску для исходного изображения после искажения перспективы
	# (нам понадобится эта маска, чтобы скопировать исходное изображение в выходное)
	# construct a mask for the source image now that the perspective
	# warp has taken place (we'll need this mask to copy the source
	# image into the destination)
	mask = np.zeros((imgH, imgW), dtype="uint8")
	cv2.fillConvexPoly(mask, dstMat.astype("int32"), (255, 255, 255),
		cv2.LINE_AA)

    # этот шаг необязателен, но чтобы сделать вокруг исходного изображения черную рамку,
	# мы можем применить операцию расширения
	# this step is optional, but to give the source image a black
	# border surrounding it when applied to the source image, you
	# can apply a dilation operation
	rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
	mask = cv2.dilate(mask, rect, iterations=2)

    # создаем трехканальную версию маски, совмещая ее по глубине,
	# чтобы мы могли скопировать искривленное исходное изображение во входное изображение
	# create a three channel version of the mask by stacking it
	# depth-wise, such that we can copy the warped source image
	# into the input image
	maskScaled = mask.copy() / 255.0
	maskScaled = np.dstack([maskScaled] * 3)

    # копируем искривленное исходное изображение во входное
    #  изображение путем (1) умножения
	# (2) умножаем оригинал входного изображение с маской
    #  (придаем больший вес входному изображению,
	# там где * НЕ * скрытые пиксели) и (3) добавляем на него результ умножения
	# copy the warped source image into the input image by
	# (1) multiplying the warped image and masked together,
	# (2) then multiplying the original input image with the
	# mask (giving more weight to the input where there
	# *ARE NOT* masked pixels), and (3) adding the resulting
	# multiplications together
	warpedMultiplied = cv2.multiply(warped.astype("float"),
		maskScaled)
	imageMultiplied = cv2.multiply(frame.astype(float),
		1.0 - maskScaled)
	output = cv2.add(warpedMultiplied, imageMultiplied)
	output = output.astype("uint8")

    # возвращаем выходной кадр
	# return the output frame to the calling function
	return output

### Создание кода дополненной реальности для видео
### Creating our OpenCV video augmented reality driver script

In [5]:

# явно укажем аргументы необходимые для работы
# input - путь к входному видеофайлу для дополненной реальности
# кэш - использовать или нет кэш опорных точек
# video - видео с шаблоном 
# output - итоговое видео 

# explicitly specify the arguments needed to work
# input - path to input video file for augmented reality
# cache - whether or not to use reference points cache
# video - video with pattern 
# output - final video 

args = {
    "input": "videos/rick_roll.mp4",
    "cache": -1,
    "video": "video.mp4",
    "output": "output.avi"
}

In [6]:
# загружаем словарь ArUCo, находим параметры и маркеры
# load the ArUCo dictionary and grab the ArUCo parameters
print("[INFO] initializing marker detector...")
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
arucoParams = cv2.aruco.DetectorParameters_create()

# инициализируем видео
# initialize the video file stream
print("[INFO] accessing video stream...")
vf = cv2.VideoCapture(args["input"])

# инициализируем последовательность кадров из видео
# initialize a queue to maintain the next frame from the video stream
Q = deque(maxlen=128)

# нам нужен кадр в последовательности, чтобы запустить нашу дополненную реальность,
# поэтому читаем следующий кадр из нашего видеофайла и добавляем
# we need to have a frame in our queue to start our augmented reality
# pipeline, so read the next frame from our video file source and add
# it to our queue
(grabbed, source) = vf.read()
Q.appendleft(source)

# инициализируем видео и указатель на выходной файл
# initialize the video stream and initialize pointer to output
# video file
print("[INFO] starting video stream...")
vs = cv2.VideoCapture(args["video"])
writer = None

[INFO] initializing marker detector...
[INFO] accessing video stream...
[INFO] starting video stream...


In [7]:
# перебираем кадры из видео
# loop over the frames from the video stream
while len(Q) > 0:
    # берем кадр из нашего видео
	# grab the frame from our video stream
	frame = vs.read()[1]

    # если кадр не был захвачен, то мы дошли до конца видео
    # if we did not grab a frame then we have reached the end of the
	# video
	if frame is None:
		break

    # изменим размер кадра, чтобы его максимальная ширина составляла 600 пикселей
    # resize the frame to have a maximum width of 600 pixels
	frame = imutils.resize(frame, width=600)

    # пытаемся найти маркеры ArUCo в кадре
	# если они найдены, берем текущее исходное изображение и деформируем его на
	# входной кадр с использованием нашей техники дополненной реальности
	# attempt to find the ArUCo markers in the frame, and provided
	# they are found, take the current source image and warp it onto
	# input frame using our augmented reality technique
	warped = find_and_warp(
		frame, source,
		cornerIDs=(923, 1001, 241, 1007),
		arucoDict=arucoDict,
		arucoParams=arucoParams,
		useCache=args["cache"] > 0)

    # если искривленный кадр существует, то мы знаем, что (1) мы нашли
	# четыре маркера ArUCo и (2) искривление перспективы успешно выполнено
	# if the warped frame is not None, then we know (1) we found the
	# four ArUCo markers and (2) the perspective warp was successfully
	# applied
	if warped is not None:
        # устанавливаем кадр в выходной кадр дополненной реальности, а затем
		# берем следующий кадр видеофайла из нашей очереди
		# set the frame to the output augment reality frame and then
		# grab the next video file frame from our queue
		frame = warped
		source = Q.popleft()

    # для скорости / эффективности мы можем использовать очередь для хранения следующего кадра
	# фокус в том, чтобы убедиться, что очередь всегда (или почти полностью) заполнена
	# for speed/efficiency, we can use a queue to keep the next video
	# frame queue ready for us -- the trick is to ensure the queue is
	# always (or nearly full)
	if len(Q) != Q.maxlen:
        # читаем следующий кадр из видеофайла
		# read the next frame from the video file stream
		(grabbed, nextFrame) = vf.read()

        # если кадр был прочитан (то есть мы не в конце
		# видеофайла), добавляем кадр в нашу очередь
		# if the frame was read (meaning we are not at the end of the
		# video file stream), add the frame to our queue
		if grabbed:
			Q.append(nextFrame)

    # если нет видеокамеры  *и* мы должны записать
    # выходное видео на диск, то инициализируем камеру
    # if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)

    # если камера работает, записываем кадр на диск
	# if the writer is not None, write the frame to disk
	if writer is not None:
		writer.write(frame)

# очищаем данные
# do a bit of cleanup
vs.release()

# проверьте, нужно ли выключить камеру
# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Выполнение ячейки может занять некоторое время. Наше выходное видео создается в формате .avi. Нам нужно преобразовать его в формат .mp4.

The execution of the cell may take some time. Our output video is created in .avi format. We need to convert it to .mp4 format.

In [8]:
!ffmpeg -i "output.avi" output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [9]:
#@title Воспроизводим выходное видео

#@title Display video inline
from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=700 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)